In [1]:
import pandas as pd
df = pd.read_csv('archive/Korea Income and Welfare.csv')
df.head()

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10


In [2]:
df.isnull().sum()

id                    0
year                  0
wave                  0
region                0
income                0
family_member         0
gender                0
year_born             0
education_level       0
marriage              0
religion              0
occupation            0
company_size          0
reason_none_worker    0
dtype: int64

In [3]:
avg_income_region = df.groupby('region')['income'].mean()
print(avg_income_region)

region
1    4028.787273
2    3907.235626
3    3433.993306
4    2771.124671
5    3742.423781
6    3081.881088
7    2870.977443
Name: income, dtype: float64


In [4]:
family_member_education = df.groupby('education_level')['family_member'].sum()
print(family_member_education)

education_level
2    15546
3    40277
4    28743
5    77809
6    18074
7    42382
8     7094
9      760
Name: family_member, dtype: int64


In [5]:
gender_religion = df.groupby(['gender', 'religion'])['marriage'].value_counts(normalize=True)
print(gender_religion)

gender  religion  marriage
1       1         1           0.887081
                  5           0.037929
                  3           0.036693
                  2           0.032549
                  4           0.005080
                  6           0.000568
                  0           0.000067
                  9           0.000033
        2         1           0.814362
                  5           0.073169
                  3           0.062030
                  2           0.040260
                  4           0.009217
                  6           0.000679
                  0           0.000226
                  9           0.000057
        9         1           0.875000
                  2           0.041667
                  5           0.041667
                  0           0.020833
                  6           0.020833
2       1         2           0.712944
                  3           0.151356
                  1           0.062563
                  5           0.04743

In [6]:
q3 = df.quantile(0.75, numeric_only=True)
q1 = df.quantile(0.25, numeric_only=True)

iqr = q3 - q1

def is_outlier(df):
    score = df['income']
    if score > q3['income'] + 1.5*iqr['income'] or score < q1['income'] - 1.5 * iqr['income']:
        return True
    else:
        return False

df['income_outlier'] = df.apply(is_outlier, axis = 1)

df = df.loc[df['income_outlier'] == False]

del df['income_outlier']

df['income'].value_counts()

3000.0    71
2400.0    70
1800.0    69
1200.0    57
840.0     56
          ..
4130.4     1
6836.0     1
3354.4     1
3706.2     1
1446.3     1
Name: income, Length: 10336, dtype: int64

In [7]:
df_job_on = df[(df['occupation'] != ' ') & (df['reason_none_worker'] == ' ') ]

df_job_on = df_job_on[(df_job_on['occupation'] != ' ') & (df_job_on['company_size'] != ' ') ]

df_job_off = df[(df['occupation'] == ' ') & (df['reason_none_worker'] != ' ') ]

df_job_on.head()

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
8,20101,2005,1,1,1257.0,1,2,1945,4,2,2,421,1,
10,20101,2007,3,1,602.0,1,2,1945,4,2,2,411,1,
11,20101,2008,4,1,1972.0,1,2,1945,4,2,2,951,1,
12,20101,2009,5,1,1638.0,1,2,1945,4,2,1,951,1,
13,20101,2010,6,1,1598.0,1,2,1945,4,2,2,951,1,


In [9]:
df_soldier = df_job_on[(df_job_on['occupation'] == '1011') | (df_job_on['occupation'] == '1012')]

df_soldier['occupation'] = '100'

soldier_num = df_job_on[(df_job_on['occupation'] == '1011') | (df_job_on['occupation'] == '1012')].index
df_job_on.drop(soldier_num, inplace = True)

In [10]:
df_job_on['occupation'] = df_job_on['occupation'].str[0] * 2

df_job_on = pd.concat([df_job_on,df_soldier])
df_job_on['occupation'].value_counts()

66    11248
99    10861
88     6961
77     5927
33     5535
22     4785
55     4351
44     3820
11     1807
Name: occupation, dtype: int64

In [15]:
df_job_off['reason_none_worker'].replace('11','0', inplace=True)

num = df_job_off[df_job_off['reason_none_worker'] == '99'].index

df_job_off.drop(num, inplace=True)
df_job_off['reason_none_worker'].value_counts()

In [19]:
df_job_off['occupation'] = df_job_off['reason_none_worker']
df_job_off['company_size'] = 0

C:\Users\wjsdl\AppData\Local\Temp\ipykernel_12308\4052709901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_job_off.loc[:, 'occupation'] = df_job_off.loc[:, 'reason_none_worker']
C:\Users\wjsdl\AppData\Local\Temp\ipykernel_12308\4052709901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_job_off.loc[:, 'company_size'] = 0


In [20]:
df1 = pd.concat([df_job_on,df_job_off])
df1.drop('reason_none_worker',axis=1,inplace=True)

In [21]:
df1.drop(['id','year'], axis = 1, inplace = True)

df1 = df1[df1['income'] >= 0]

In [22]:
num1 = df1[(df1['marriage'] == 0) | (df1['marriage'] == 9)].index
df1.drop(num1, inplace =True)

num2 = df1[df1['religion'] == 9].index
df1.drop(num2, inplace=True)

num3 = df1[df1['company_size']=='99'].index
df1.drop(num3, inplace=True)

In [23]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86972 entries, 8 to 92848
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   wave             86972 non-null  int64  
 1   region           86972 non-null  int64  
 2   income           86972 non-null  float64
 3   family_member    86972 non-null  int64  
 4   gender           86972 non-null  int64  
 5   year_born        86972 non-null  int64  
 6   education_level  86972 non-null  int64  
 7   marriage         86972 non-null  int64  
 8   religion         86972 non-null  int64  
 9   occupation       86972 non-null  object 
 10  company_size     86972 non-null  object 
dtypes: float64(1), int64(8), object(2)
memory usage: 8.0+ MB


In [17]:
df1['region'] = df['region'].apply(str)

df1['company_size'] = pd.to_numeric(df1['company_size'])

df1['gender'].replace([1,2],[0,1],inplace=True)
df1['religion'].replace([1,2],[0,1],inplace=True)

In [18]:
cal_cols = df1.select_dtypes('object').columns.values

df2 = pd.get_dummies(data=df1,columns=cal_cols)

In [19]:
X = df2.drop('income',axis=1).values
y = df2['income'].values

X.shape, y.shape

((87231, 36), (87231,))